# DataFrame Methods More II

This chapter covers several more useful but not as common DataFrame methods and operations.

* `drop_duplicates`
* `nunique`
* addition and multiplication with string columns
* `clip`
* `nlargest/nsmallest`
* `copy`
* `insert`
* `pop`
* `corr`
* `replace`

## Dropping duplicate rows with `drop_duplicates`
Previously, we learned about the `unique` method which only exists for Series objects. DataFrames have a method called `drop_duplicates` that functions similarly. The default call to the `drop_duplicates` method returns only unique rows of the DataFrame. It does not use the index value in its search for duplicates. If two or more rows are duplicated, the first row is kept. Let's see if there are any duplicate rows in the employee dataset.

In [ ]:
import pandas as pd
emp = pd.read_csv('../data/employee.csv', parse_dates=['hire_date'])
emp.head()

In [ ]:
emp.shape

In [ ]:
emp.drop_duplicates().shape

Interestingly, there are some rows with the exact same information for all six columns.

### Drop duplicates based on a subset of columns
Instead of dropping rows where the entire row is duplicated, you can restrict the search for duplication to a subset of the columns. Pass the `subset` parameter a single column name or a list of column names to keep rows where the first unique appearance occurs. The following example returns a single row for each unique department.

In [ ]:
emp.drop_duplicates(subset='dept')

Return the first row for each unique combination of race and gender.

In [ ]:
emp.drop_duplicates(subset=['race', 'gender'])

## The `nunique` method
The `nunique` method returns the number of unique values for each column. It technically is an aggregation method as it returns a single value for each column. 

In [ ]:
emp.nunique()

The `nunique` method does not count missing values as unique by default. Set the `dropna` parameter to `False` if you'd like to include a count (of at most one) for missing values.

In [ ]:
emp.nunique(dropna=False)

## Addition and multiplication with string columns
Addition actually does work with strings by appending the word being added to each value. You can also use multiplication to concatenate each string value to itself.

In [ ]:
(emp.select_dtypes('object') + 'SOMESTRING').head()

In [ ]:
(emp.select_dtypes('object') * 3).head()

## The `clip` method

The `clip` method works only on numeric columns and can be used to limit values to certain thresholds. It works by taking a `lower` and `upper` bound. Any value below/above the lower/upper bound is changed so that it is exactly that number. This method works for both Series and DataFrames. We use it below to bound the salary column.

In [ ]:
sal_head = emp['salary'].head(10)
sal_head

Bound the salary to 50,000 and 100,000.

In [ ]:
sal_head.clip(50000, 100000)

It's not required to have bounds both the lower and upper values. Bounding the values on one side is possible by passing in just one of the parameters.

In [ ]:
sal_head.clip(upper=100000)

## The `nlargest/nsmallest` methods
The `nlargest` and `nsmallest` methods provide a similar solution that `sort_values` does. Pass them `n`, the number of rows you want to return and `columns`, a string of a column name you would like the largest or smallest values from. The following returns the employees with the 5 highest salaries.

In [ ]:
emp.nlargest(5, 'salary')

It is possible to duplicate this with `sort_values` together with the `head` method.

In [ ]:
emp.sort_values('salary', ascending=False).head()

### Why use `nlargest/nsmallest`?
While `nlargest/nsmallest` can be duplicated with `sort_values`, in theory, `nlargest/nsmallest` should perform better as it uses a [selection algorithm][1] and not a sorting one. The `nlargest/nsmallest` methods also have the ability to keep the top n rows with ties with the `keep` parameter. 

[1]: https://en.wikipedia.org/wiki/Selection_algorithm

## Copying a DataFrame

The `copy` method is available to make a completely new copy of a DataFrame that is not associated with the original. This is necessary because assigning a DataFrame to a new variable does not copy it. Let's read in a sample DataFrame and assign it to different variable names.

In [ ]:
df = pd.read_csv('../data/sample_data.csv', index_col=0)
df

We can create a new variable by assigning it to the `df` DataFrame. This does not make a new copy of the data.

In [ ]:
df1 = df
df1

If you are unfamiliar with Python, you might make the mistake assuming that `df` and `df1` reference different DataFrames. What we have is a single DataFrame object that is referenced by two different variable names. We can prove this with the `is` operator.

In [ ]:
df is df1

We can also show this by modifying `df` by adding a new column and then viewing `df1`.

In [ ]:
df['new_col'] = 5
df1

The variables `df` and `df1` are just two different names that reference the same underlying DataFrame. If you'd like to create a completely new DataFrame with the same data, you need to use the `copy` method. Let's reread in the same dataset again, but this time assign `df1` to a copy of `df`.

In [ ]:
df = pd.read_csv('../data/sample_data.csv', index_col=0)
df1 = df.copy()

Testing whether `df` and `df1` reference the same DataFrame will result now yield `False`.

In [ ]:
df is df1

If we add a column to `df` it will have no effect on `df1`.

In [ ]:
df['new_col'] = 5
df1

## Inserting columns in the middle of a DataFrame

We previously learned about adding a new column to a DataFrame using just the brackets. By default, new columns are placed at the end of the DataFrame. Let's read in a few of the columns from the `college` DataFrame. You can use the `usecols` parameter to specify a subset of the columns to read in the `read_csv` function.

In [ ]:
cols = ['instnm', 'relaffil', 'satvrmid', 'satmtmid', 'distanceonly', 'ugds']
college = pd.read_csv('../data/college.csv', index_col='instnm', usecols=cols)
college.head()

Let's create a new column equal to the total SAT score using just the brackets.

In [ ]:
college['sat_total'] = college['satvrmid'] + college['satmtmid']
college.head()

With this method, there is no option to specify the position of the new column. It is always appended to the end. If you desire to place the column in a different location, you need to use the `insert` method instead. This method has three required parameters:
* `loc` - the integer location of the new column
* `column` - the name of the new column
* `value` - the values of the new column

This method works **in-place** and is one of the only ones that does so by default. This means that the calling DataFrame gets modified and there is nothing that is returned. There is no assignment statement when using `insert`. Let's insert the same SAT total right after the `satmtmid` column. We will call it `sat_total_insert` to differentiate it from the column on the end.

In [ ]:
vals = college['satvrmid'] + college['satmtmid']
college.insert(3, 'sat_total_insert', vals)
college.head()

One minor annoyance is that you must know the integer location of where you'd like to insert the new column. In the above example, its easy-enough to just count, but a more automated solution would be nice. The pandas Index object has a method called `get_loc` which returns the integer location of a column name. 

This is a rare instance in this book where an Index method is used. I advise not digging into Index objects unless there is some very specialized need. So, with some hesitation, I present the `get_loc` Index method here. First, access the `columns` attribute (which is an Index object) and pass the `get_loc` method the name the column.

In [ ]:
college.columns.get_loc('satmtmid')

Make note that the `get_loc` method does not exist for Series or DataFrame objects. It is strictly an Index method available to either the index or the columns.

### Comparison to Python lists
The DataFrame `insert` method is analogous to a Python list method with the same name. It too inserts a value into the list in-place given an integer location. Let's complete an example to compare how it works.

In [ ]:
a = ['some', 'list', 'of', 'strings']
a

Call the list `insert` method which mutates the list in-place.

In [ ]:
a.insert(1, 'short')
a

There's also an `index` method that returns the integer location of a particular item in the list which is analogous to the `get_loc` method.

In [ ]:
a.index('of')

## The `pop` method

The DataFrame `pop` method will remove a single column from a DataFrame and return it as a Series. This is different than the `drop` method which removes a column or columns and returns a new DataFrame of the remaining columns. The `pop` method modifies the calling DataFrame in-place. Below, we remove the `ugds` column and assign it to a variable with the same name.

In [ ]:
ugds = college.pop('ugds')
ugds.head()

The `college` DataFrame no longer contains the `ugds` column.

In [ ]:
college.head()

## The `corr` method

The `corr` method computes the correlation between every pair of numeric columns in the DataFrame. The resulting DataFrame is 'square' meaning it has the same number of rows as it does columns. The row names and column names are the same. Also, the DataFrame is 'symmetric' meaning that taking the transpose results in the exact same DataFrame.

In [ ]:
college.corr().round(2)

Notice that pandas silently drops the non-numeric columns. Also, each variable has a perfect correlation of 1 to itself. The default correlation method is 'pearson' but 'kendall' and 'spearman' are also available. The above result is rounded to two decimals to help clean up its appearance.

## The `replace` method

The `replace` method can be used to replace values in your DataFrame. It is very powerful and flexible. It is also quite complex as there are many different combinations of parameters to handle a variety of different kinds of replacement. Let's use the head of the employee DataFrame to see some examples. We use the `round` method to round salaries to the nearest 10,000.

In [ ]:
emp_head = emp.head().round(-4)
emp_head

The `replace` method has two main parameters, `to_replace` and `value`. The simplest application is to set each one to a single value. Below, we replace all of the values equal to 70,000 with 99,999. All values in the entire DataFrame are searched to be replaced.

In [ ]:
emp_head.replace(to_replace=70000, value=99999)

The `replace` method can also replace exact strings.

In [ ]:
emp_head.replace(to_replace='Male', value='GENDER IS MALE')

You can also use a dictionary to map the old value to the new value. When using a dictionary, you do not use the parameter `value`. Below, we replace 'White' with 'RACE IS WHITE'.

In [ ]:
emp_head.replace(to_replace={'White':'RACE IS WHITE'})

You can replace as many values as you'd like with a dictionary.

In [ ]:
emp_head.replace(to_replace={'White':'WHITE', 50000: 11111, 70000: 99999})

### Replacing Substrings
It is possible to replace a portion of a string and not the entire string as was done above. In order to do so you'll need to set the `regex` parameter to `True`. By default, `regex` is set to `False`. Let's first see an example that attempts to replace the word 'Department' with 'dept' but doesn't actually do anything.

In [ ]:
emp_head.replace({'Department':'dept'})

This does nothing because pandas is looking for the exact value 'Department'. The word 'Department' does appear in many strings but it is not the only word in the string. Let's run the same command but set `regex` to `True`.

In [ ]:
emp_head.replace({'Department':'dept'}, regex=True)

### What is `regex`?
The parameter `regex` stands for **regular expression** which is a huge topic on its own and several chapters are dedicated to it later in the book. A regular expression matches patterns within text. Our pattern from above is very simple and is just the literal characters 'Department'. To show just one example (without explanation) of the power of regex, let's replace every occurrence of the character 'o' or 'O' with 'ZZZ'.

In [ ]:
emp_head.replace({'[oO]':'ZZZ'}, regex=True)

There is more to the `replace` method, but these examples are how it is primarily used. It's probably easiest to just use a dictionary as shown in the last few examples.

## Finding the maximum/minimum of a group
In this section, we will explore a practical example of the `drop_duplicates` method. Let's say we are interested in finding the employee with the maximum salary per department. The resulting DataFrame will have a single row for each department. Let's begin by sorting by department first and then salary making sure salary is sorted from greatest to least.

In [ ]:
emp_sorted = emp.sort_values(['dept', 'salary'], ascending=[True, False])
emp_sorted.head()

The data is correctly sorted, but the information we want is not easily accessible. We desire a single row for each department. We can now turn to the `drop_duplicates` method and use the `subset` parameter to keep the first row of every department.

In [ ]:
emp_sorted.drop_duplicates(subset='dept')

We can rewrite our solution without assigning the result of `sort_values` to a variable by chaining the `drop_duplicates` method directly after it. Here, we find the employee with the lowest salary per department.

In [ ]:
emp.sort_values(['dept', 'salary']) \
   .drop_duplicates(subset='dept')

It's actually sufficient to sort just by salary as the first value encountered for each department will be the employee with the highest salary. Note, how the final data will be sorted by salary and not by department.

In [ ]:
emp.sort_values('salary', ascending=False) \
   .drop_duplicates(subset='dept')

This short chain of steps combining `sort_values` with `drop_duplicates` is a generic and common pattern for finding the maximum or minimum of some column within groups formed by other columns. Below, we find the minimum salary for every unique combination of race and gender.

In [ ]:
emp.sort_values('salary') \
   .drop_duplicates(subset=['race', 'gender'])

## Exercises

Use the employee dataset for the first few problems.

### Exercise 1
<span  style="color:green; font-size:16px">How many unique combinations of department and title exist?</span>

### Exercise 2
<span  style="color:green; font-size:16px">Since only Series methods have a `unique` method, can you think of a creative way of getting the same result as Exercise 1 with the `unique` method?</span>

### Exercise 3
<span  style="color:green; font-size:16px">Find the occurrence of all race and gender combinations. For instance, you would return an object that contains the number of 'Hispanic Males', 'Black Females', etc...</span>

### Exercise 4

<span  style="color:green; font-size:16px">Find the relative frequency of departments for all employees and then find the relative frequency of departments for the top 100 salaries. Compare the differences.</span>

### Exercise 5

<span  style="color:green; font-size:16px">Create a new column `bonus` right after the salary column equal to 10% of the salary. Round the bonus to the nearest thousand.</span>

### Use the flights dataset for the remaining exercises.

In [ ]:
flights = pd.read_csv('../data/flights.csv')
pd.set_option('display.max_columns', 40)
flights.head()

### Exercise 6

<span  style="color:green; font-size:16px">Find the five variables with the highest correlation to `departure_delay`.</span>

### Exercise 7

<span  style="color:green; font-size:16px">Replace the exact string 'LAX' with 'Los Angeles Airport', 'IAH' with 'George Bush Airport', 'SLC' with 'Salt Lake City Airport', and 'UA' with 'United Airlines'.</span>

### Exercise 8

<span  style="color:green; font-size:16px">Find the largest departure delay for every airline. Return just these two columns.</span>